In [ ]:
# DO NOT MODIFY THIS CELL
# Imports libraries, to install, use 'pip install <package name>
import pandas as pd
import numpy as np
import re
from pandas.api.types import is_numeric_dtype

# Defines Functions
# Drops the index Column
def drop_index(df):
    if is_numeric_dtype(df.iloc[:, 0]):
        return (df.iloc[:, 1:])
    else:
        return df

# Cleans Phone Numbers
def clean_phone(number):
    # Convert to string and keep only digits
    digits = ''.join([char for char in str(number) if char.isdigit()])
    
    # Check if exactly 10 digits
    if len(digits) == 10:
        return f'({digits[:3]}){digits[3:6]}-{digits[6:]}'
    else:
        return np.nan

# Extracts emails
def extract_email(text):
    text = str(text)
    i = 0
    at_pos = text.find('@', i)
    if at_pos == -1:
        return np.nan
    dot_pos = text.find('.', at_pos)
    if dot_pos == -1:
        return np.nan
    if(dot_pos+4>len(text)):
        return np.nan
    return(text[0:(dot_pos+4)])

# Extracts contractor name, dropping anything with brackets
def extract_contractor(text):
    text = str(text)
    parts = text.split('(')
    for part in parts:
        return part
    return np.nan
    
# Extracts and cleans the address, correcting 'San Jose' spelled as 'San Josh'
def extract_address(text):
    text = pd.Series([text])
    new_text = text.str.replace("san josh", "San Jose", case = False, regex = True)
    new_text = new_text.str.replace('\n', '', regex = False)
    new_text = new_text.str.replace(r'\s*,\s*', ',', regex=True)
    new_text = new_text.str.replace(',', ', ', regex = False)
    return new_text

# Renames columns
def rename_columns_by_index(df, rename_dict):
    new_columns = df.columns.tolist()
    for idx, new_name in rename_dict.items():
        if 0 <= idx < len(new_columns):
            new_columns[idx] = new_name
    df.columns = new_columns
    return df

# Drops any data without valid phones or emials
def drop_rows_if_columns_blank(df, col_idx1, col_idx2):
    df = df.copy()
    # Replace empty strings with NaN for accurate detection
    df.replace('', pd.NA, inplace=True)
    
    # Drop rows where both specified columns are blank/NaN
    mask = df.iloc[:, col_idx1].isna() & df.iloc[:, col_idx2].isna()
    return df[~mask]

# Cleans Duplicates
def clean_duplicates(df):

    phone_dup = df.iloc[:,2].duplicated()
    df['phone_dup'] = phone_dup
    df = pd.concat([df, phone_dup])
    email_dup = df.iloc[:,3].duplicated() 
    df['email_dup'] = phone_dup
    df = pd.concat([df, email_dup])
    df = df[df['phone_dup']==False]
    df = df[df['email_dup']==False]
    df = df.drop(columns = 'phone_dup')
    df = df.drop(columns = 'email_dup')
    return df


In [ ]:
# Additional Functions for Information about resulting data
# DO NOT MODIFY THIS CELL
# Prints business name frequency information
def find_names(df):
    total = df.shape[0]
    names = df.iloc[:,0]
    name_list = []
    for name in names:
        name_list.append(name.split())
    name_list_ind = []
    for i in name_list:
        for j in i:
            name_list_ind.append(j)
    name_list_values = {}

    # Find item frequencies dict
    for c in name_list_ind:
        #  in dictionary
        if c in name_list_values:
            name_list_values[c] += 1
        #  not in dictionary
        else:
            name_list_values[c] = 1

    # removes bad values
    bad_vals = ["Inc", 'Inc.', '&', 'Valley', 'Hill', 'Palo', 'Alto', 'SF', 'Morgan', 'Bay', 'Golden', 'Experts', 'Silicon', 'Gate', 'Area', 'Peninsula', 'and', 'Group', 'Crest', 'Co.', 'south']
    for i in bad_vals:
        if i in name_list_values:
            del name_list_values[i]
    
    sorted_items = sorted(name_list_values.items(), key=lambda kv: (kv[1], kv[0]), reverse = True)
    
    # Finds percentages
    #total = sum(count for word, count in sorted_items)
    # Convert to percentages
    percentages = [(word, count / total * 100) for word, count in sorted_items]
    # Round 
    percentages = [(word, round(pct, 2)) for word, pct in percentages]
    
    print('percentage of times each word occurs')
    print(percentages)
    print('number of times each word comes up')
    print (sorted_items)

# Prints locational information
def find_location(df):
    names = df.iloc[:,1]
    name_list = []
    for name in names:
        name_list.append(name.split())
    name_list_ind = []
    for i in name_list:
        for j in i:
            name_list_ind.append(j)
    name_list_values = {}

    # Find item frequencies dict
    for c in name_list_ind:
        #  in dictionary
        if c in name_list_values:
            name_list_values[c] += 1
        #  not in dictionary
        else:
            name_list_values[c] = 1

    # removes bad values
    bad_vals = ['CA', 'San', 'St,', 'Ave', 'Blvd', 'Ave,', 'Box,', 'PO', 'Box']
    for i in bad_vals:
        if i in name_list_values:
            del name_list_values[i]
    
    sorted_items = sorted(name_list_values.items(), key=lambda kv: (kv[1], kv[0]), reverse = True)

    print(sorted_items)


In [ ]:
# Calls All functions
# DO NOT MODIFY THIS CELL
def data_cleaner(df, column_names, phone_column =2, email_column = 3):
    clean_data = df.copy()
    # cleans data
    clean_data = drop_index(clean_data)
    clean_data.iloc[:, 2] = clean_data.iloc[:, 2].apply(clean_phone)
    clean_data.iloc[:, 3] = clean_data.iloc[:, 3].apply(extract_email)
    clean_data.iloc[:, 0] = clean_data.iloc[:, 0].apply(extract_contractor)
    clean_data.iloc[:, 1] = clean_data.iloc[:, 1].apply(extract_address)
    clean_data = rename_columns_by_index(clean_data, column_names)
    clean_data = drop_rows_if_columns_blank(clean_data,phone_column,email_column)
    clean_data = clean_duplicates(clean_data)
    return clean_data

In [ ]:
# Imports Excel Spreadsheet
# To import a new sheet, make sure it is in the same folder as this file
# Make sure the columns are in the order: Index(optional), customer name, address, phone number, email
# CHANGE THIS LINE
df = pd.read_excel('contact_info1.xlsx')

In [ ]:
# Cleans data
# Change variables as needed

# Column names of the outputed sheet, feel free to change them as needed
column_names = {0:'Customer_Name', 1: 'Address', 2: 'Phone_Number', 3:'Email' }

# Calls function, and displays results, DO NOT CHANGE LINES BELOW
clean_data = df.copy()
clean_data = data_cleaner(clean_data, column_names)

# Prints out the Sheet information, and head
print("Sheet columns, rows:", clean_data.shape)
clean_data.head()

In [ ]:
# Downloads the data as a CSV, and Excel. 
# Change the names inside the quotes to change the name of the file that will be created
# This will not work if a file with the intended name already exists.
# It should say download Complete if the new files where created
clean_contacts = clean_data.to_csv('clean_contacts.csv', index=False)
clean_contacts_xlx = clean_data.to_excel('clean_contacts.xlsx', index=False)
print('download complete')

In [ ]:
# Optional: Gives some additional information about the sheet clean data
find_names(clean_data)
find_location(clean_data)